In [4]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import requests
import json
import tls_client
from unidecode import unidecode
import time

In [6]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

requests = tls_client.Session(
    client_identifier="chrome112",
)

response1 = requests.get('https://api.prizepicks.com/projections')

prizepicks = response1.json()

In [11]:
df = pd.json_normalize(prizepicks,record_path='data')
df

,type,id,attributes.board_time,attributes.custom_image,attributes.description,attributes.discount_percentage,attributes.end_time,attributes.flash_sale_line_score,attributes.is_promo,attributes.line_score,...,relationships.league.data.type,relationships.league.data.id,relationships.new_player.data.type,relationships.new_player.data.id,relationships.projection_type.data.type,relationships.projection_type.data.id,relationships.stat_type.data.type,relationships.stat_type.data.id,relationships.duration.data.type,relationships.duration.data.id
0,projection,1607576,2023-10-03T15:00:00-04:00,https://imgur.com/biyDVTG.png,PHI,18.0,2023-10-03T18:30:00-04:00,4.5,True,5.5,...,league,2,new_player,69805,projection_type,2,stat_type,311,NaN,NaN
1,projection,1404500,2023-06-21T23:00:00-04:00,None,2023-2024 Season,NaN,None,NaN,False,6.8,...,league,173,new_player,53241,projection_type,2,stat_type,395,NaN,NaN
2,projection,1607129,2023-10-01T15:00:00-04:00,None,LVA,NaN,None,NaN,False,3.5,...,league,3,new_player,141538,projection_type,2,stat_type,20,NaN,NaN
3,projection,1607462,2023-08-27T14:00:00-04:00,None,J. Sinner (Beijing),NaN,None,NaN,False,9.5,...,league,244,new_player,94976,projection_type,2,stat_type,325,NaN,NaN
4,projection,1607461,2023-08-27T14:00:00-04:00,None,J. Sinner (Beijing),NaN,None,NaN,False,9.5,...,league,244,new_player,94976,projection_type,2,stat_type,324,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,projection,1605698,2023-10-02T17:04:24-04:00,None,Imperial,NaN,None,NaN,False,9.0,...,league,124,new_player,20096,projection_type,2,stat_type,337,NaN,NaN
877,projection,1605699,2023-10-02T17:04:24-04:00,None,Imperial,NaN,None,NaN,False,16.5,...,league,124,new_player,78129,projection_type,2,stat_type,337,NaN,NaN
878,projection,1605700,2023-10-02T17:04:24-04:00,None,Imperial,NaN,None,NaN,False,18.0,...,league,124,new_player,62441,projection_type,2,stat_type,337,NaN,NaN
879,projection,1605701,2023-10-02T17:04:24-04:00,None,Imperial,NaN,None,NaN,False,17.0,...,league,124,new_player,51044,projection_type,2,stat_type,337,NaN,NaN


In [13]:
df2 = pd.json_normalize(prizepicks,record_path='included')
df2.head(3)

,type,id,attributes.combo,attributes.display_name,attributes.image_url,attributes.league,attributes.league_id,attributes.market,attributes.name,attributes.position,...,relationships.league.data.id,attributes.lfg_ignored_leagues,attributes.rank,attributes.active,attributes.f2p_enabled,attributes.icon,attributes.last_five_games_enabled,attributes.league_icon_id,attributes.projections_count,relationships.projection_filters.data
0,new_player,162368,False,Tyus Jones,https://static.prizepicks.com/images/players/n...,NBASZN,173.0,None,Tyus Jones,PG,...,173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,new_player,42794,False,prosus,https://i.imgur.com/pn5ljs6.png,CSGO,124.0,None,prosus,G,...,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,stat_type,44,NaN,NaN,NaN,NaN,NaN,NaN,Pass TDs,NaN,...,NaN,[],6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df2.drop(df2.columns[df2.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [15]:
#getting names dict from df2
display(df2.head())
names = df2[['type','id', 'attributes.name']]
names = names.loc[names['type']=='new_player']
names = names[['id','attributes.name']]

name = names['attributes.name'].tolist()
id = names['id'].tolist()

names_dict = {id[i]: name[i] for i in range(len(name))}

names_dict

,type,id,attributes.combo,attributes.display_name,attributes.image_url,attributes.league,attributes.league_id,attributes.market,attributes.name,attributes.position,...,relationships.league.data.id,attributes.lfg_ignored_leagues,attributes.rank,attributes.active,attributes.f2p_enabled,attributes.icon,attributes.last_five_games_enabled,attributes.league_icon_id,attributes.projections_count,relationships.projection_filters.data
0,new_player,162368,False,Tyus Jones,https://static.prizepicks.com/images/players/n...,NBASZN,173.0,None,Tyus Jones,PG,...,173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,new_player,42794,False,prosus,https://i.imgur.com/pn5ljs6.png,CSGO,124.0,None,prosus,G,...,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,stat_type,44,NaN,NaN,NaN,NaN,NaN,NaN,Pass TDs,NaN,...,NaN,[],6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,new_player,151733,False,Sergio Canales,https://s3-us-west-2.amazonaws.com/static.fant...,SOCCER,82.0,CF Monterrey,Sergio Canales,Midfielder,...,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,new_player,76146,False,Brad Marchand,https://static.prizepicks.com/images/players/n...,NHLSZN,236.0,None,Brad Marchand,F,...,236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


{'162368': 'Tyus Jones',
 '42794': 'prosus',
 '151733': 'Sergio Canales',
 '76146': 'Brad Marchand',
 '143530': 'Victor Wembanyama',
 '4259': 'Cristian Garin',
 '149134': 'Jace Peterson',
 '79555': 'Bam Adebayo',
 '147612': 'Jake Burger',
 '76172': 'Jacob Markstrom',
 '150697': 'Terry McLaurin',
 '76117': 'Auston Matthews',
 '15712': 'Jin Hyuk No',
 '150690': 'Brock Purdy',
 '149934': 'Pablo González',
 '170164': 'Filip Ugrinic',
 '152548': 'Dameon Pierce',
 '150232': 'Federico Bernardeschi',
 '150564': 'André-Pierre Gignac',
 '149442': 'Luis Romo',
 '150406': 'Emanuel Gularte',
 '150402': 'Federico Lértora',
 '3357': ' Grant Dawson',
 '49940': 'b0denmaster',
 '150530': 'Julián Quiñones',
 '76068': 'Connor Hellebuyck',
 '169923': 'malibu',
 '70925': 'Choi Hyoung-woo',
 '15692': 'Sang-su Kim',
 '150667': 'Amon-Ra St. Brown',
 '160308': 'Asier Illarramendi',
 '87419': 'Josh Giddey',
 '169926': 'dodonut',
 '70923': 'Park Chan-ho',
 '150684': 'Bijan Robinson',
 '15405': 'PlesseN',
 '160951

In [16]:
df.head()

,type,id,attributes.board_time,attributes.custom_image,attributes.description,attributes.discount_percentage,attributes.end_time,attributes.flash_sale_line_score,attributes.is_promo,attributes.line_score,...,relationships.league.data.type,relationships.league.data.id,relationships.new_player.data.type,relationships.new_player.data.id,relationships.projection_type.data.type,relationships.projection_type.data.id,relationships.stat_type.data.type,relationships.stat_type.data.id,relationships.duration.data.type,relationships.duration.data.id
0,projection,1607576,2023-10-03T15:00:00-04:00,https://imgur.com/biyDVTG.png,PHI,18.0,2023-10-03T18:30:00-04:00,4.5,True,5.5,...,league,2,new_player,69805,projection_type,2,stat_type,311,NaN,NaN
1,projection,1404500,2023-06-21T23:00:00-04:00,None,2023-2024 Season,NaN,None,NaN,False,6.8,...,league,173,new_player,53241,projection_type,2,stat_type,395,NaN,NaN
2,projection,1607129,2023-10-01T15:00:00-04:00,None,LVA,NaN,None,NaN,False,3.5,...,league,3,new_player,141538,projection_type,2,stat_type,20,NaN,NaN
3,projection,1607462,2023-08-27T14:00:00-04:00,None,J. Sinner (Beijing),NaN,None,NaN,False,9.5,...,league,244,new_player,94976,projection_type,2,stat_type,325,NaN,NaN
4,projection,1607461,2023-08-27T14:00:00-04:00,None,J. Sinner (Beijing),NaN,None,NaN,False,9.5,...,league,244,new_player,94976,projection_type,2,stat_type,324,NaN,NaN


In [17]:
#viewing all the columns to see what i need

base = df[:-1]
base.loc[15]

type                                                      projection
id                                                           1606415
attributes.board_time                      2023-04-05T19:50:00-04:00
attributes.custom_image                                         None
attributes.description                                Mauricio Ruffy
attributes.discount_percentage                                   NaN
attributes.end_time                                             None
attributes.flash_sale_line_score                                 NaN
attributes.is_promo                                            False
attributes.line_score                                           35.5
attributes.projection_type                               Single Stat
attributes.rank                                                    1
attributes.refundable                                           True
attributes.start_time                      2023-10-03T20:10:00-04:00
attributes.stat_type              

In [18]:
id[0:5]

['162368', '42794', '151733', '76146', '143530']

In [19]:
ids = df['relationships.new_player.data.id'].unique()
print(len(ids))


606


In [20]:
#trimming excess columns and adding actual player names

df = base[['attributes.description','attributes.line_score',
        'attributes.stat_type','relationships.new_player.data.id','relationships.league.data.id','attributes.start_time']]

df['Player'] = df['relationships.new_player.data.id'].map(names_dict)
df = df[['Player','attributes.description', 'attributes.line_score', 'attributes.stat_type','relationships.league.data.id','attributes.start_time']]

df = df.rename(columns={"attributes.description": "Team", "attributes.line_score": "Line", 
                        'attributes.stat_type':'Stat','relationships.league.data.id':'League',
                       'attributes.start_time':'Date'})

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_11852\2042246692.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Player'] = df['relationships.new_player.data.id'].map(names_dict)


In [21]:
df.League.unique()

array(['2', '173', '3', '244', '9', '12', '193', '236', '5', '82', '231',
       '159', '8', '135', '124'], dtype=object)

In [22]:
full = df.copy(deep=True)

In [23]:
#df = df.loc[df.League=='7']
df['Date'] = df['Date'].apply(lambda x: x.split('T')[0])
df['Player'] = df['Player'].apply(lambda x: unidecode(x).replace('_',' '))
df

,Player,Team,Line,Stat,League,Date
0,Jesus Luzardo,PHI,5.5,Pitcher Strikeouts,2,2023-10-03
1,Lebron James,2023-2024 Season,6.8,Assists Per Game AVG,173,2023-10-24
2,Breanna Stewart,LVA,3.5,Assists,3,2023-10-08
3,Daniil Medvedev,J. Sinner (Beijing),9.5,2nd Set - Total Games,244,2023-10-04
4,Daniil Medvedev,J. Sinner (Beijing),9.5,1st Set - Total Games,244,2023-10-04
...,...,...,...,...,...,...
875,LUCAS1,Party Astronauts,17.0,MAPS 1-2 Headshots,124,2023-10-04
876,PwnAlone,Imperial,9.0,MAPS 1-2 Headshots,124,2023-10-04
877,Grizz,Imperial,16.5,MAPS 1-2 Headshots,124,2023-10-04
878,spek,Imperial,18.0,MAPS 1-2 Headshots,124,2023-10-04


In [24]:
syntax = {
    'Points': 'PTS',
    'Rebs': 'TRB',
    'Asts': 'AST',
    'Rebounds': 'TRB',
    '3-PT Made': '3P',
    'Assists': 'AST',
    'Turnovers': 'TOV',
    'Blks': 'BLK',
    'Pts': 'PTS',
    'Pts+Asts': 'PTS+AST',
    'Pts+Rebs+Asts':'PTS+TRB+AST',
    'Pts+Rebs':'PTS+TRB',
    'Rebs+Asts':'TRB+AST',
    'Blks+Stls':'BLK+STL',
    'Stls': 'STL',
    'Free Throws Made': 'FT',
    'Blocked Shots': 'BLK',
    'Steals': 'STL',
    'Personal Fouls': 'PF',
    'FG Attempted': 'FGA'
}
df = df.replace(syntax)
df.Stat.unique()

array(['Pitcher Strikeouts', 'Assists Per Game AVG', 'AST',
       '2nd Set - Total Games', '1st Set - Total Games', 'Pass TDs',
       'Fantasy Score', 'Fight Time (Mins)', 'Pass Yards',
       '3rd Set - Total Games', 'Points Per Game AVG', 'Takedowns', 'PTS',
       'Rebounds Per Game AVG', 'Significant Strikes', 'TRB', 'Goals',
       'Pass+Rush+Rec TDs', 'Receiving Yards', 'Aces', 'Total Games',
       'Break Points Won', 'Rush Yards', 'Receptions',
       'Home Runs (Combo)', 'Shots', 'Tackles', 'Shots Assisted',
       'Goalie Saves', 'Passes Attempted', 'Clearances',
       'Shots On Target', 'Goalie Saves (Combo)', 'INT', 'Pass+Rush Yds',
       'Goals Allowed in First 30 Minutes', 'Pitches Thrown',
       'Hits Allowed', 'Runs Allowed', 'Kicking Points', 'FG Made',
       '3PT Made Per Game AVG', 'Hitter Fantasy Score',
       'Hits+Runs+RBIs (Combo)', 'Total Bases', '1st Inning Runs Allowed',
       'Hits+Runs+RBIS', 'Hitter Strikeouts', 'Pitching Outs',
       'Pitcher Fant

In [25]:
len(df)

880

In [26]:
df = df[df.Stat != 'Fantasy Score']

In [27]:
len(df)

844

In [28]:
df = df.replace('Fred VanVleet\t','Fred VanVleet')
df = df.replace('Nicolas Claxton','Nic Claxton')
df = df.replace('Robert Williams III','Robert Williams')
df = df.replace('LeBron James\t','LeBron James')
df= df.replace('KJ Martin Jr.','Kenyon Martin Jr.')
df= df.replace('Xavier Tillman','Xavier Tillman Sr.')#xav

In [29]:
df = df.reset_index(drop=True)

In [30]:
today= dt.date.today()
df.sample(5)

,Player,Team,Line,Stat,League,Date
542,Kyle Schwarber,MIA,1.5,Hitter Strikeouts,2,2023-10-03
708,Casey Kelly,Lotte Giants,6.0,Hits Allowed,135,2023-10-04
385,Pablo Lopez,TOR 4th Inning,0.5,Hits Allowed,231,2023-10-03
555,Corbin Burnes,ARI,4.0,Hits Allowed,2,2023-10-03
47,Kelsey Plum,NYL 1st Half,9.0,PTS,193,2023-10-08


In [31]:
f'lines{today.month}_{today.day}'

'lines10_3'

In [32]:
nba = df.loc[df.League=='7']
nba.to_csv(f'Lines/lines{today.month}_{today.day}')

In [33]:
pd.option_context('display.max_columns', 5)
df

,Player,Team,Line,Stat,League,Date
0,Jesus Luzardo,PHI,5.5,Pitcher Strikeouts,2,2023-10-03
1,Lebron James,2023-2024 Season,6.8,Assists Per Game AVG,173,2023-10-24
2,Breanna Stewart,LVA,3.5,AST,3,2023-10-08
3,Daniil Medvedev,J. Sinner (Beijing),9.5,2nd Set - Total Games,244,2023-10-04
4,Daniil Medvedev,J. Sinner (Beijing),9.5,1st Set - Total Games,244,2023-10-04
...,...,...,...,...,...,...
839,LUCAS1,Party Astronauts,17.0,MAPS 1-2 Headshots,124,2023-10-04
840,PwnAlone,Imperial,9.0,MAPS 1-2 Headshots,124,2023-10-04
841,Grizz,Imperial,16.5,MAPS 1-2 Headshots,124,2023-10-04
842,spek,Imperial,18.0,MAPS 1-2 Headshots,124,2023-10-04


In [34]:
df['Date'].max()

'2023-10-25'

In [52]:
df['time'] = dt.datetime.now().replace(microsecond=0,second=0)
df

,Player,Team,Line,Stat,League,Date,time
0,Jesus Luzardo,PHI,5.5,Pitcher Strikeouts,2,2023-10-03,2023-10-03 16:52:00
1,Lebron James,2023-2024 Season,6.8,Assists Per Game AVG,173,2023-10-24,2023-10-03 16:52:00
2,Breanna Stewart,LVA,3.5,AST,3,2023-10-08,2023-10-03 16:52:00
3,Daniil Medvedev,J. Sinner (Beijing),9.5,2nd Set - Total Games,244,2023-10-04,2023-10-03 16:52:00
4,Daniil Medvedev,J. Sinner (Beijing),9.5,1st Set - Total Games,244,2023-10-04,2023-10-03 16:52:00
...,...,...,...,...,...,...,...
839,LUCAS1,Party Astronauts,17.0,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00
840,PwnAlone,Imperial,9.0,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00
841,Grizz,Imperial,16.5,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00
842,spek,Imperial,18.0,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00


In [53]:
df.to_csv(f'Lines/full_lines{today.month}_{today.day}')

In [54]:
old = pd.read_csv('Lines/full_lines10_3',index_col=0)

In [55]:
fake_new = old.copy(deep=True)

In [56]:
fake_new

,Player,Team,Line,Stat,League,Date,time
0,Jesus Luzardo,PHI,5.5,Pitcher Strikeouts,2,2023-10-03,2023-10-03 16:52:00
1,Lebron James,2023-2024 Season,6.8,Assists Per Game AVG,173,2023-10-24,2023-10-03 16:52:00
2,Breanna Stewart,LVA,3.5,AST,3,2023-10-08,2023-10-03 16:52:00
3,Daniil Medvedev,J. Sinner (Beijing),9.5,2nd Set - Total Games,244,2023-10-04,2023-10-03 16:52:00
4,Daniil Medvedev,J. Sinner (Beijing),9.5,1st Set - Total Games,244,2023-10-04,2023-10-03 16:52:00
...,...,...,...,...,...,...,...
839,LUCAS1,Party Astronauts,17.0,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00
840,PwnAlone,Imperial,9.0,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00
841,Grizz,Imperial,16.5,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00
842,spek,Imperial,18.0,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00


In [58]:
final = pd.concat((old,fake_new))

In [59]:
final.loc[final.Player =='Grizz']

,Player,Team,Line,Stat,League,Date,time
831,Grizz,Imperial,32.5,MAPS 1-2 Kills,124,2023-10-04,2023-10-03 16:52:00
841,Grizz,Imperial,16.5,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00
831,Grizz,Imperial,32.5,MAPS 1-2 Kills,124,2023-10-04,2023-10-03 16:52:00
841,Grizz,Imperial,16.5,MAPS 1-2 Headshots,124,2023-10-04,2023-10-03 16:52:00
